In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
# 加载数据
data_path = 'HUAWEI.csv'  # 替换为您的文件路径
new_data = pd.read_csv(data_path)

# 准备数据
data_tensor = torch.tensor(new_data['forecast'].values.astype(np.float32)).view(-1, 1)
dataset = TensorDataset(data_tensor, data_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 定义自编码器模型
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(1, 2),
            nn.ReLU())
        self.decoder = nn.Sequential(
            nn.Linear(2, 1),
            nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# 初始化模型、损失函数和优化器
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 50
for epoch in range(num_epochs):
    for data in dataloader:
        inputs, _ = data
        outputs = model(inputs)
        loss = criterion(outputs, inputs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# 评估模型
model.eval()
with torch.no_grad():
    predictions = model(data_tensor)
    mse = nn.functional.mse_loss(predictions, data_tensor, reduction='none')
    mse = mse.view(-1).numpy()

new_data['Reconstruction_Error'] = mse

# 定义重建误差的阈值为离群点
threshold = np.percentile(new_data['Reconstruction_Error'], 95)
new_data['Outlier_Autoencoder'] = new_data['Reconstruction_Error'] > threshold

# 使用seaborn样式
sns.set(style="whitegrid")

# 创建折线图，并在其中标注离群点
plt.figure(figsize=(12, 6))

# 绘制forecast值的折线图
plt.plot(new_data['forecast'], label='Forecast Value', color='blue')

# 标注离群点
outliers = new_data[new_data['Outlier_Autoencoder']]
plt.scatter(outliers.index, outliers['forecast'], color='red', label='Outliers (Autoencoder)')

# 设置标题和标签
plt.title('Forecast Value with Autoencoder Outliers', fontsize=15, fontweight='bold')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Forecast Value', fontsize=12)
plt.legend()

# 显示图表
plt.show()